In [1]:
import time
import numpy as np
#import cupy as np
#coudb't appy the code to the gpu using cupy I'll use other ways
import sys
sys.path.append("../src")
from logistic_regression.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from logistic_regression.data_loader import load_and_resize_images, build_label_array
from logistic_regression.model import model
from logistic_regression.eval_utils import print_report
from logistic_regression.visualization import plot_costs

In [2]:
TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000
LR = 0.001

## Load and parse XML labels

In [3]:
labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

## Load and preprocess images

In [4]:
X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)


## Flatten Inputs

In [5]:
train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

(12288, 2612) (1, 2612)
(12288, 327) (1, 327)


## Defining the neural network structure

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [7]:
def layer_sizes(X, Y, n_h=128):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    n_h -- number of nodes in the hidden layer
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    #(≈ 3 lines of code)
    # n_x = ... 
    # n_h = ...
    # n_y = ... 
    # YOUR CODE STARTS HERE
    n_x = np.shape(X)[0]
    n_y= np.shape(Y)[0]
    
    # YOUR CODE ENDS HERE
    return (n_x, n_h, n_y)

In [22]:
def initialize_parameters(n_x, n_h, n_y, seed=3):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """    
    #(≈ 4 lines of code)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # YOUR CODE STARTS HERE
    np.random.seed(seed)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    
    # YOUR CODE ENDS HERE

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [9]:

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    #(≈ 4 lines of code)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # YOUR CODE STARTS HERE
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    
    # YOUR CODE ENDS HERE
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    # (≈ 4 lines of code)
    # Z1 = ...
    # A1 = ...
    # Z2 = ...
    # A2 = ...
    # YOUR CODE STARTS HERE
    
    Z1 = np.dot(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
     
    # YOUR CODE ENDS HERE
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [10]:

def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    
    m = Y.shape[1] # number of examples


    # Compute the cross-entropy cost
    # (≈ 2 lines of code)
    # logprobs = ...
    # cost = ...
    # YOUR CODE STARTS HERE
    eps = 1e-8
    first_half = np.dot(Y,np.log(A2 + eps).T)
    socend_half= np.dot((1-Y),np.log(1-A2 + eps).T)
    cost= first_half + socend_half
    cost/=-m
    # YOUR CODE ENDS HERE
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    
    return cost

In [11]:

def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    #(≈ 2 lines of code)
    # W1 = ...
    # W2 = ...
    # YOUR CODE STARTS HERE
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    # YOUR CODE ENDS HERE
        
    # Retrieve also A1 and A2 from dictionary "cache".
    #(≈ 2 lines of code)
    # A1 = ...
    # A2 = ...
    # YOUR CODE STARTS HERE
    A1 = cache["A1"]
    A2 = cache["A2"]
        
    # YOUR CODE ENDS HERE
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    #(≈ 6 lines of code, corresponding to 6 equations on slide above)
    # dZ2 = ...
    # dW2 = ...
    # db2 = ...
    # dZ1 = ...
    # dW1 = ...
    # db1 = ...
    # YOUR CODE STARTS HERE
    dZ2 = A2 - Y 
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2,axis=1,keepdims= True)/m
    dZ1 = np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m
    db1 = np.sum(dZ1,axis=1,keepdims= True)/m
    # YOUR CODE ENDS HERE
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [12]:
import copy

def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve a copy of each parameter from the dictionary "parameters". Use copy.deepcopy(...) for W1 and W2
    #(≈ 4 lines of code)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # YOUR CODE STARTS HERE
    W1 = copy.deepcopy(parameters["W1"])
    b1 = copy.deepcopy(parameters["b1"])
    W2 = copy.deepcopy(parameters["W2"])
    b2 = copy.deepcopy(parameters["b2"])
    
    # YOUR CODE ENDS HERE
    
    # Retrieve each gradient from the dictionary "grads"
    #(≈ 4 lines of code)
    # dW1 = ...
    # db1 = ...
    # dW2 = ...
    # db2 = ...
    # YOUR CODE STARTS HERE
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # YOUR CODE ENDS HERE
    
    # Update rule for each parameter
    #(≈ 4 lines of code)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # YOUR CODE STARTS HERE
    W1 += -learning_rate*dW1
    b1 += -learning_rate*db1
    W2 += -learning_rate*dW2
    b2 += -learning_rate*db2
    
    # YOUR CODE ENDS HERE
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [13]:
def predict(parameters, X):
    """
    Predict using the trained 2-layer network.
    Returns Y_prediction (1, m) with 0/1 values.
    """
    A2, _ = forward_propagation(X, parameters)
    Y_prediction = (A2 > 0.5).astype(int)
    return Y_prediction


In [14]:
def accuracy(predictions, Y):
    return float(np.mean(predictions == Y) * 100)

## nn_model

In [21]:

def nn_model(X_train, Y_train, n_h=128, num_iterations = 10000, learning_rate=1.2, print_cost=False, X_test=None, Y_test=None):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X_train, Y_train)[0]
    n_y = layer_sizes(X_train, Y_train)[2]
    
    # Initialize parameters
    #(≈ 1 line of code)
    # parameters = ...
    # YOUR CODE STARTS HERE
    (n_x, n_h, n_y) = layer_sizes(X_train, Y_train)
    parameters = initialize_parameters(n_x, n_h, n_y)

    
    # YOUR CODE ENDS HERE
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        #(≈ 4 lines of code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        # A2, cache = ...
        
        # Cost function. Inputs: "A2, Y". Outputs: "cost".
        # cost = ...
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        # grads = ...
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        # parameters = ...
        
        # YOUR CODE STARTS HERE
        A2, cache = forward_propagation(X_train, parameters)

        cost = compute_cost(A2, Y_train)
        
        grads = backward_propagation(parameters, cache, X_train, Y_train)

        parameters = update_parameters(parameters, grads, learning_rate)
        # YOUR CODE ENDS HERE
        
        # Print the cost every 1000 iterations
        if print_cost and (i % 1000 == 0):
            msg = f"Cost after iteration {i}: {cost:.6f}"
            if X_test is not None and Y_test is not None:
                train_pred = predict(parameters, X_train)
                test_pred = predict(parameters, X_test)
                train_acc = accuracy(train_pred, Y_train)
                test_acc = accuracy(test_pred, Y_test)
                msg += f" | train acc: {train_acc:.2f}% | test acc: {test_acc:.2f}%"
            print(msg)
    if X_test is not None and Y_test is not None:
        train_pred = predict(parameters, X_train)
        test_pred = predict(parameters, X_test)
        train_acc = accuracy(train_pred, Y_train)
        test_acc = accuracy(test_pred, Y_test)
        print(f"Final train accuracy: {train_acc:.2f}%")
        print(f"Final test accuracy:  {test_acc:.2f}%")

    return parameters

In [20]:
params = nn_model(train_x, Y_train_org,
                  n_h=128,
                  num_iterations=10000,
                  learning_rate=1.2,
                  print_cost=True,
                  X_test=test_x,
                  Y_test=Y_test_org)

ImportError: libcurand.so.10: cannot open shared object file: No such file or directory